In [2]:
import pandas as pd
import numpy as np

In [3]:
# 读取数据
train_data = pd.read_csv('./house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('./house-prices-advanced-regression-techniques/test.csv')

In [4]:
# 去除ID
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))   # 将所有的训练数据和测试数据的79个特征按样本连结。

In [5]:
# 标准化
# 我们对连续数值的特征做标准化（standardization）：设该特征在整个数据集上的均值为 μ ，标准差为 σ 。
# 那么，我们可以将该特征的每个值先减去 μ 再除以 σ 得到标准化后的每个特征值。对于缺失的特征值，我们将其替换成该特征的均值。
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index   # 得到数值类型索引
all_features[numeric_features] = all_features[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))     # 数值类型标准化方便填缺失数据            
# 标准化后，每个特征的均值变为0，所以可以直接用0来替换缺失值
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [6]:
# 离散特征转化
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

(2919, 331)

In [7]:
# 这一步转换将特征数从79增加到了331。
# 最后，通过values属性得到NumPy格式的数据，并转成ndarray方便后面的训练。
n_train = train_data.shape[0]
train_features = np.array(all_features[:n_train].values)
test_features = np.array(all_features[n_train:].values)
train_labels = np.array(train_data.SalePrice.values).reshape((-1, 1))
print(np.max(test_features))
print(np.max(train_features))
print(np.max(train_labels))
# print(np.max(all_features[numeric_features]))
print(train_labels)

29.871533309269406
27.227905963254084
755000
[[208500]
 [181500]
 [223500]
 ...
 [266500]
 [142125]
 [147500]]


In [39]:
# 预测标签集的数据使用log缩减，防止平方产生误差过大而溢出
train_labels = np.log(train_labels)
print(np.max(test_features))
print(np.max(train_features))
print(np.max(train_labels))

29.871533309269406
27.227905963254084
13.534473028231162


In [40]:
# 至此我们数据处理完毕
print(test_features.shape, train_features.shape, train_labels.shape)
print(train_labels)

(1459, 331) (1460, 331) (1460, 1)
[[12.24769432]
 [12.10901093]
 [12.31716669]
 ...
 [12.49312952]
 [11.86446223]
 [11.90158345]]


In [109]:
# 划分训练时的测试集和训练集
train_ = np.random.choice(range(len(train_features)), 800, replace=False)
test_ = np.array([i for i in range(len(train_features)) if i not in train_])
train_features_ = train_features[train_]
train_labels_ = train_labels[train_]
test_features_ = train_features[test_]
test_labels_ = train_labels[test_]

In [42]:
# 计算预测数据正确率
def right_rate(pred_label, label):
    '''
    input:
    pred_label:预测的结果数组
    label:实际的类别数组
    output:
    rate:正确率
    '''
    diff = pred_label - label
    diff = (diff<1)
    return diff.tolist().count(True) / len(diff)

In [11]:
# 全连接层的初始化类
class FullyConnected():
    def __init__(self, W, b):
        '''
        Parameter:
        W: 权重矩阵，形状为(N, M), N为输入神经元的个数，M为输出神经元的个数
        b: 偏移量矩阵 (M，)
        '''
        self.W = W    # 赋值
        self.b = b    # 赋值

        self.x = None      # 用来存储输入神经元的矩阵，为反向传播提供便利

        self.dW = None   # 用来存储梯度，梯度下降时需要用来更新权重
        self.db = None
    
    # 全连接层的前向传播
    def forward(self, x):
        '''
        input:
        x: 输入神经元的矩阵, 形状(B, N)，B为批量大小，N为输入神经元的个数
        output:
        y: 输出神经元的矩阵, 形状(B, M)， M为输出神经元的个数
        '''
        self.x = x            # 存储输入神经元的矩阵，便于反向传播计算更新权重
        out = np.dot(self.x, self.W) + self.b   # 完成一次前向传播
        return out    # 返回前向传播结果
    
    
    # 全连接层的反向传播
    def backward(self, dout):
        '''
        input:
        dout: 损失函数相对于全连接层输出的梯度，形状为(B,M)，M是全连接层的输出神经元个数。
        在前向传播时全连接层的输入记录在了self.x中，故由此我们可以利用dout和self.x得到W的梯度
        output:
        dx:  (B, N) 关于输入层的梯度，便于进一步反向传播
        self.W和self.b的梯度分别存储在self.dW和self.db中
        self.dW: (N, M) 与self.W形状相同，self.W的梯度
        self.db: (M,)， self.b的梯度
        将x的梯度返回。
        '''
        # 以下所有即为矩阵的求导方法，我们也可以根据形状输入输出求解
        # 均依赖于公式 Y = X^T + W
        self.db = np.sum(dout, axis=0)    # 需要将得到的所有dout延y轴相加，因为取loss是就除以了batch_size
        self.dW = np.dot(self.x.T, dout)  # 在前向传播时全连接层的输入记录在了self.x中，这一项根据矩阵求导得到我们结果
        dx = np.dot(dout, self.W.T)       # 由矩阵求导得出结果
        return dx                       # 返回对输入层求导的结果，便于记录进一步反向传播

In [12]:
# 激活函数的初始化类
class Sigmoid():
    '''
    Parameter:
    z:激活函数作用后得到的矩阵
    '''
    def __init__(self):
        self.z = None
    
    
    # Sigmoid激活函数的前向传播。
    def forward(self, y):
        '''
        input: 
        y:全连接层前向传播得到矩阵，形状为(B, N)
        output:
        z:激活函数作用后得到矩阵，形状为(B, N)
        '''
        z = np.exp(y) / (1 + np.exp(y)) # 利用np.exp直接对矩阵运算
        self.z = z   # 赋值
        return z  # 返回矩阵
    
    # sigmoid的反向传播
    def backward(self, dout):
        '''
        input: 
        dout：损失函数相对于sigmoid输出的梯度
        output:
        dz:相对于矩阵y得到的梯度
        '''
        dz = dout * self.z * (np.ones(self.z.shape) - self.z)
        return dz

class Relu:
    def __init__(self):
        '''
        Parameter:
        z:激活函数作用后得到的矩阵
        '''
        self.mask = None
    
    # Relu激活函数的前向传播。
    def forward(self, y):
        '''
        input: 
        y:全连接层前向传播得到矩阵，形状为(B, N)
        output:
        z:激活函数作用后得到矩阵，形状为(B, N)
        '''
        self.mask = (y <= 0)   # 得到关于y大于小于0的真值的矩阵
        z = y.copy()       # 深度拷贝一个y矩阵
        z[self.mask] = 0   # 将小于零的值赋为0
        return z   # 返回矩阵

    def backward(self, dout):
        '''
        input: 
        dout：损失函数相对于relu输出的梯度
        output:
        dz:相对于矩阵y得到的梯度
        '''
        dout[self.mask] = 0
        dz = dout
        return dz

In [13]:
# 误差损失函数定义（均方误差）
class MeanSquaredError:
    def __init__(self):
        '''
        Parameter:
        y : 预测输出结果矩阵，利用其求出误差损失，形状为(B, 1)
        label: 真实标签矩阵，形状为 (B， 1)
        '''
        self.loss = None
        self.z = None
        self.label = None
    
    # Mean Squared Error的前向传播
    def forward(self, y, label):
        '''
        input:
        y : 预测输出结果矩阵，利用其求出误差损失，形状为(B, 10)
        label: 真实标签矩阵，形状为 (B， 1)
        output:
        loss: 均方误差损失
        '''
        loss = 0.5 * np.sum((y - label) ** 2)
        self.loss = loss
        self.z = y
        self.label = label
        return loss
    
    # Mean Squared Error的反向传播
    def backward(self):
        '''
        output:
        均方误差梯度
        '''
        return self.z - self.label

In [54]:
# 神经网络实现
class Network:
    # 初始化
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size=1, lr=0.1):
        '''
        Parameters:
        input_size, hidden_size1, hidden_size2, output_size:
        分别为输入层神经元个数、隐藏层神经元数、隐藏层神经元个数、输出层神经元个数数(手写数字识别默认为10, 回归默认为1), 学习率(默认为0.1)
        output:None
        '''
        W1 = np.random.randn(input_size, hidden_size1)  # 随机初始化权重
        W2 = np.random.randn(hidden_size1, hidden_size2)
        W3 = np.random.randn(hidden_size2, output_size)
        b1 = np.random.randn(hidden_size1)
        b2 = np.random.randn(hidden_size2)
        b3 = np.random.randn(output_size)

        
        self.lr = lr     # 学习率
        self.layer_1 = FullyConnected(W1, b1)
        self.activ_1 = Sigmoid()
        self.layer_2 = FullyConnected(W2, b2)
        self.activ_2 = Sigmoid()
        self.layer_last = FullyConnected(W3, b3)
        self.loss = MeanSquaredError()
    
    # 神经网络前向传播
    def forward(self, x, label):
        '''
        input:
        x: 形状为(B,N)，输入的原始数据， B为批量Batch_size
        label:输入B个数据的分类类别，形状为(B, 1)
        output:
        最后输出的预测向量以及我们得到的误差
        '''
        y1 = self.layer_1.forward(x)  # 前向传播，一步步往后走
        z1 = self.activ_1.forward(y1)
        y2 = self.layer_2.forward(z1)
        z2 = self.activ_2.forward(y2)
        y3 = self.layer_last.forward(z2)
        loss = self.loss.forward(y3, label)
        
        return y3, loss
    
    
    # 神经网络反向传播
    def backward(self):
        '''
        input:None
        output:
        各项参数的梯度
        '''
        d = self.loss.backward()     # 反向传播，一步步往前走，和前向完全相反
        d = self.layer_last.backward(d)
        d = self.activ_2.backward(d)
        d = self.layer_2.backward(d)
        d = self.activ_1.backward(d)
        d = self.layer_1.backward(d)    # 至此，我们单次反向传播完成。
        
        return self.layer_1.dW, self.layer_1.db, self.layer_2.dW, self.layer_2.db, self.layer_last.dW, self.layer_last.db  # 将每层间权重的W 、偏移量b梯度返回
        # 这一步只是为了方便检查展示，并没有很大的用途


    # 神经网络更新权重
    def refresh(self):
        lr = self.lr
        self.layer_1.W -= lr * self.layer_1.dW
        self.layer_1.b -= lr * self.layer_1.db
        self.layer_2.W -= lr * self.layer_2.dW
        self.layer_2.b -= lr * self.layer_2.db
        self.layer_last.W -= lr * self.layer_last.dW
        self.layer_last.b -= lr * self.layer_last.db
    
    # 训练模型并判断正确率
    def fit_pred(self, train_images, train_labels, test_images, test_labels, Epochs=5, batch_size=100):
        '''
        input：
        train_images:训练集
        test_labels:测试集
        output:None
        '''
        samples_num = train_images.shape[0]   # 得到训练集数量
        for epoch in range(1, Epochs + 1):   # 在训练集里面跑5次
            i = 0
            while i < samples_num:
                self.forward(train_images[i:i+batch_size], train_labels[i:i+batch_size])  # 每次训练batch_size个样本
                self.backward()         # 反向传播
                self.refresh()          # 更新参数
#                 print("Train Epoch: {}\t batch_size_index:{} Loss:{:.6f}".format(epoch, i+1, self.loss.loss))
                i += batch_size
                
#             self.lr = (0.95 ** epoch) * self.lr     # 更新学习率，防止其因为学习率过大而导致无法有效下降。
            print("Train Epoch: {}\t Loss:{:.6f}".format(epoch, self.loss.loss))
            pred, pred_loss, right_rate = self.predict(test_images, test_labels)    # 计算测试集精度
            print("Test -- Average loss:{:.4f}, Accuracy:{:.3f}\n".format(pred_loss, right_rate))
    
    # 检验测试集的模型
    def predict(self, test_images, test_labels):
        '''
        '''
        pred, loss = self.forward(test_images, test_labels)  # 预测值和损失
#         pred = np.argmax(pred, axis=1)   # 求出预测标签
        return pred, loss, right_rate(pred, test_labels)   # 返回预测值向量和损失误差以及正确率。
    
    # 预测函数
    def predict_(self, test_feature):
        '''
        '''
        x = test_feature
        y1 = self.layer_1.forward(x)  # 前向传播，一步步往后走
        z1 = self.activ_1.forward(y1)
        y2 = self.layer_2.forward(z1)
        z2 = self.activ_2.forward(y2)
        y3 = self.layer_last.forward(z2)
        return y3

In [93]:
a = Network(331, 200, 20, lr=0.001)
a.fit_pred(train_features_, train_labels_, test_features_, test_labels_ , Epochs=200, batch_size=50)

Train Epoch: 1	 Loss:35.506818
Test -- Average loss:179.8452, Accuracy:0.000

Train Epoch: 2	 Loss:26.010604
Test -- Average loss:127.7595, Accuracy:0.000

Train Epoch: 3	 Loss:19.734489
Test -- Average loss:98.5138, Accuracy:0.000

Train Epoch: 4	 Loss:15.149646
Test -- Average loss:80.8138, Accuracy:0.000

Train Epoch: 5	 Loss:11.768205
Test -- Average loss:68.9342, Accuracy:0.000

Train Epoch: 6	 Loss:9.290038
Test -- Average loss:60.4579, Accuracy:0.000

Train Epoch: 7	 Loss:7.504852
Test -- Average loss:54.2147, Accuracy:0.000

Train Epoch: 8	 Loss:6.237954
Test -- Average loss:49.5025, Accuracy:0.000

Train Epoch: 9	 Loss:5.326126
Test -- Average loss:45.8486, Accuracy:0.000

Train Epoch: 10	 Loss:4.650996
Test -- Average loss:42.9410, Accuracy:0.000

Train Epoch: 11	 Loss:4.136631
Test -- Average loss:40.5783, Accuracy:0.000

Train Epoch: 12	 Loss:3.734547
Test -- Average loss:38.6315, Accuracy:0.000

Train Epoch: 13	 Loss:3.412670
Test -- Average loss:37.0176, Accuracy:0.000

T

Train Epoch: 111	 Loss:1.558875
Test -- Average loss:19.9169, Accuracy:0.000

Train Epoch: 112	 Loss:1.556813
Test -- Average loss:19.8753, Accuracy:0.000

Train Epoch: 113	 Loss:1.554712
Test -- Average loss:19.8341, Accuracy:0.000

Train Epoch: 114	 Loss:1.552575
Test -- Average loss:19.7935, Accuracy:0.000

Train Epoch: 115	 Loss:1.550402
Test -- Average loss:19.7535, Accuracy:0.000

Train Epoch: 116	 Loss:1.548194
Test -- Average loss:19.7139, Accuracy:0.000

Train Epoch: 117	 Loss:1.545952
Test -- Average loss:19.6749, Accuracy:0.000

Train Epoch: 118	 Loss:1.543678
Test -- Average loss:19.6363, Accuracy:0.000

Train Epoch: 119	 Loss:1.541372
Test -- Average loss:19.5982, Accuracy:0.000

Train Epoch: 120	 Loss:1.539035
Test -- Average loss:19.5606, Accuracy:0.000

Train Epoch: 121	 Loss:1.536668
Test -- Average loss:19.5235, Accuracy:0.000

Train Epoch: 122	 Loss:1.534274
Test -- Average loss:19.4868, Accuracy:0.000

Train Epoch: 123	 Loss:1.531852
Test -- Average loss:19.4505, Ac

In [94]:
b = Network(331, 200, 10, lr=0.001)
b.fit_pred(train_features_, train_labels_, test_features_, test_labels_ , Epochs=200, batch_size=50)

Train Epoch: 1	 Loss:26.448699
Test -- Average loss:107.3563, Accuracy:0.000

Train Epoch: 2	 Loss:19.253262
Test -- Average loss:82.7304, Accuracy:0.000

Train Epoch: 3	 Loss:14.959695
Test -- Average loss:71.0684, Accuracy:0.000

Train Epoch: 4	 Loss:12.740850
Test -- Average loss:64.0235, Accuracy:0.000

Train Epoch: 5	 Loss:11.067153
Test -- Average loss:58.8994, Accuracy:0.000

Train Epoch: 6	 Loss:9.575523
Test -- Average loss:54.9304, Accuracy:0.000

Train Epoch: 7	 Loss:8.455833
Test -- Average loss:51.7250, Accuracy:0.000

Train Epoch: 8	 Loss:7.737049
Test -- Average loss:49.1255, Accuracy:0.000

Train Epoch: 9	 Loss:7.267125
Test -- Average loss:47.0065, Accuracy:0.000

Train Epoch: 10	 Loss:6.927375
Test -- Average loss:45.2441, Accuracy:0.000

Train Epoch: 11	 Loss:6.656219
Test -- Average loss:43.7522, Accuracy:0.000

Train Epoch: 12	 Loss:6.423167
Test -- Average loss:42.4671, Accuracy:0.000

Train Epoch: 13	 Loss:6.213187
Test -- Average loss:41.3390, Accuracy:0.000

Tr

Test -- Average loss:18.6026, Accuracy:0.000

Train Epoch: 111	 Loss:2.416565
Test -- Average loss:18.5524, Accuracy:0.000

Train Epoch: 112	 Loss:2.405598
Test -- Average loss:18.5032, Accuracy:0.000

Train Epoch: 113	 Loss:2.394870
Test -- Average loss:18.4550, Accuracy:0.000

Train Epoch: 114	 Loss:2.384375
Test -- Average loss:18.4076, Accuracy:0.000

Train Epoch: 115	 Loss:2.374106
Test -- Average loss:18.3610, Accuracy:0.000

Train Epoch: 116	 Loss:2.364055
Test -- Average loss:18.3151, Accuracy:0.000

Train Epoch: 117	 Loss:2.354217
Test -- Average loss:18.2701, Accuracy:0.000

Train Epoch: 118	 Loss:2.344584
Test -- Average loss:18.2257, Accuracy:0.000

Train Epoch: 119	 Loss:2.335150
Test -- Average loss:18.1819, Accuracy:0.000

Train Epoch: 120	 Loss:2.325908
Test -- Average loss:18.1388, Accuracy:0.000

Train Epoch: 121	 Loss:2.316851
Test -- Average loss:18.0963, Accuracy:0.000

Train Epoch: 122	 Loss:2.307974
Test -- Average loss:18.0544, Accuracy:0.000

Train Epoch: 123	 

In [95]:
c = Network(331, 100, 20, lr=0.001)
c.fit_pred(train_features_, train_labels_, test_features_, test_labels_ , Epochs=200, batch_size=50)

Train Epoch: 1	 Loss:26.699077
Test -- Average loss:163.8731, Accuracy:0.000

Train Epoch: 2	 Loss:16.182069
Test -- Average loss:106.5461, Accuracy:0.000

Train Epoch: 3	 Loss:11.546566
Test -- Average loss:78.9729, Accuracy:0.000

Train Epoch: 4	 Loss:8.946866
Test -- Average loss:63.1002, Accuracy:0.000

Train Epoch: 5	 Loss:7.304912
Test -- Average loss:52.8450, Accuracy:0.000

Train Epoch: 6	 Loss:6.198565
Test -- Average loss:45.7220, Accuracy:0.000

Train Epoch: 7	 Loss:5.417919
Test -- Average loss:40.5267, Accuracy:0.000

Train Epoch: 8	 Loss:4.845702
Test -- Average loss:36.5965, Accuracy:0.000

Train Epoch: 9	 Loss:4.412424
Test -- Average loss:33.5363, Accuracy:0.000

Train Epoch: 10	 Loss:4.075168
Test -- Average loss:31.0965, Accuracy:0.000

Train Epoch: 11	 Loss:3.806437
Test -- Average loss:29.1130, Accuracy:0.000

Train Epoch: 12	 Loss:3.587998
Test -- Average loss:27.4738, Accuracy:0.000

Train Epoch: 13	 Loss:3.407373
Test -- Average loss:26.1001, Accuracy:0.000

Tra

Test -- Average loss:12.5591, Accuracy:0.000

Train Epoch: 111	 Loss:1.185095
Test -- Average loss:12.5366, Accuracy:0.000

Train Epoch: 112	 Loss:1.174565
Test -- Average loss:12.5145, Accuracy:0.000

Train Epoch: 113	 Loss:1.164116
Test -- Average loss:12.4927, Accuracy:0.000

Train Epoch: 114	 Loss:1.153752
Test -- Average loss:12.4712, Accuracy:0.000

Train Epoch: 115	 Loss:1.143477
Test -- Average loss:12.4499, Accuracy:0.000

Train Epoch: 116	 Loss:1.133295
Test -- Average loss:12.4289, Accuracy:0.000

Train Epoch: 117	 Loss:1.123210
Test -- Average loss:12.4082, Accuracy:0.000

Train Epoch: 118	 Loss:1.113225
Test -- Average loss:12.3878, Accuracy:0.000

Train Epoch: 119	 Loss:1.103343
Test -- Average loss:12.3677, Accuracy:0.000

Train Epoch: 120	 Loss:1.093568
Test -- Average loss:12.3478, Accuracy:0.000

Train Epoch: 121	 Loss:1.083903
Test -- Average loss:12.3281, Accuracy:0.000

Train Epoch: 122	 Loss:1.074350
Test -- Average loss:12.3087, Accuracy:0.000

Train Epoch: 123	 

In [97]:
d = Network(331, 100, 50, lr=0.001)
d.fit_pred(train_features_, train_labels_, test_features_, test_labels_ , Epochs=400, batch_size=50)

Train Epoch: 1	 Loss:58.379878
Test -- Average loss:311.2622, Accuracy:0.000

Train Epoch: 2	 Loss:39.922085
Test -- Average loss:238.9209, Accuracy:0.000

Train Epoch: 3	 Loss:31.482894
Test -- Average loss:198.1549, Accuracy:0.000

Train Epoch: 4	 Loss:26.409493
Test -- Average loss:170.3322, Accuracy:0.000

Train Epoch: 5	 Loss:22.854046
Test -- Average loss:149.6271, Accuracy:0.000

Train Epoch: 6	 Loss:20.140095
Test -- Average loss:133.4545, Accuracy:0.000

Train Epoch: 7	 Loss:17.959182
Test -- Average loss:120.4153, Accuracy:0.000

Train Epoch: 8	 Loss:16.147698
Test -- Average loss:109.6561, Accuracy:0.000

Train Epoch: 9	 Loss:14.610102
Test -- Average loss:100.6157, Accuracy:0.000

Train Epoch: 10	 Loss:13.286757
Test -- Average loss:92.9065, Accuracy:0.000

Train Epoch: 11	 Loss:12.138154
Test -- Average loss:86.2512, Accuracy:0.000

Train Epoch: 12	 Loss:11.136343
Test -- Average loss:80.4466, Accuracy:0.000

Train Epoch: 13	 Loss:10.260061
Test -- Average loss:75.3401, Ac

Test -- Average loss:13.6328, Accuracy:0.000

Train Epoch: 116	 Loss:1.434092
Test -- Average loss:13.5762, Accuracy:0.000

Train Epoch: 117	 Loss:1.427121
Test -- Average loss:13.5207, Accuracy:0.000

Train Epoch: 118	 Loss:1.420274
Test -- Average loss:13.4663, Accuracy:0.000

Train Epoch: 119	 Loss:1.413548
Test -- Average loss:13.4128, Accuracy:0.000

Train Epoch: 120	 Loss:1.406939
Test -- Average loss:13.3603, Accuracy:0.000

Train Epoch: 121	 Loss:1.400444
Test -- Average loss:13.3088, Accuracy:0.000

Train Epoch: 122	 Loss:1.394060
Test -- Average loss:13.2582, Accuracy:0.000

Train Epoch: 123	 Loss:1.387784
Test -- Average loss:13.2085, Accuracy:0.000

Train Epoch: 124	 Loss:1.381614
Test -- Average loss:13.1597, Accuracy:0.000

Train Epoch: 125	 Loss:1.375545
Test -- Average loss:13.1117, Accuracy:0.000

Train Epoch: 126	 Loss:1.369577
Test -- Average loss:13.0646, Accuracy:0.000

Train Epoch: 127	 Loss:1.363705
Test -- Average loss:13.0183, Accuracy:0.000

Train Epoch: 128	 

Test -- Average loss:10.5171, Accuracy:0.000

Train Epoch: 228	 Loss:1.035489
Test -- Average loss:10.5033, Accuracy:0.000

Train Epoch: 229	 Loss:1.033642
Test -- Average loss:10.4897, Accuracy:0.000

Train Epoch: 230	 Loss:1.031809
Test -- Average loss:10.4762, Accuracy:0.000

Train Epoch: 231	 Loss:1.029991
Test -- Average loss:10.4628, Accuracy:0.000

Train Epoch: 232	 Loss:1.028186
Test -- Average loss:10.4495, Accuracy:0.000

Train Epoch: 233	 Loss:1.026396
Test -- Average loss:10.4364, Accuracy:0.000

Train Epoch: 234	 Loss:1.024620
Test -- Average loss:10.4233, Accuracy:0.000

Train Epoch: 235	 Loss:1.022857
Test -- Average loss:10.4104, Accuracy:0.000

Train Epoch: 236	 Loss:1.021108
Test -- Average loss:10.3976, Accuracy:0.000

Train Epoch: 237	 Loss:1.019372
Test -- Average loss:10.3849, Accuracy:0.000

Train Epoch: 238	 Loss:1.017649
Test -- Average loss:10.3723, Accuracy:0.000

Train Epoch: 239	 Loss:1.015939
Test -- Average loss:10.3598, Accuracy:0.000

Train Epoch: 240	 

Train Epoch: 336	 Loss:0.891552
Test -- Average loss:9.5205, Accuracy:0.000

Train Epoch: 337	 Loss:0.890559
Test -- Average loss:9.5147, Accuracy:0.000

Train Epoch: 338	 Loss:0.889569
Test -- Average loss:9.5089, Accuracy:0.000

Train Epoch: 339	 Loss:0.888583
Test -- Average loss:9.5031, Accuracy:0.000

Train Epoch: 340	 Loss:0.887602
Test -- Average loss:9.4974, Accuracy:0.000

Train Epoch: 341	 Loss:0.886624
Test -- Average loss:9.4917, Accuracy:0.000

Train Epoch: 342	 Loss:0.885650
Test -- Average loss:9.4861, Accuracy:0.000

Train Epoch: 343	 Loss:0.884680
Test -- Average loss:9.4805, Accuracy:0.000

Train Epoch: 344	 Loss:0.883714
Test -- Average loss:9.4749, Accuracy:0.000

Train Epoch: 345	 Loss:0.882751
Test -- Average loss:9.4694, Accuracy:0.000

Train Epoch: 346	 Loss:0.881793
Test -- Average loss:9.4639, Accuracy:0.000

Train Epoch: 347	 Loss:0.880837
Test -- Average loss:9.4584, Accuracy:0.000

Train Epoch: 348	 Loss:0.879886
Test -- Average loss:9.4530, Accuracy:0.000


In [112]:
e = Network(331, 100, 80, lr=0.001)
e.fit_pred(train_features_, train_labels_, test_features_, test_labels_ , Epochs=400, batch_size=50)

Train Epoch: 1	 Loss:38.883343
Test -- Average loss:735.4439, Accuracy:0.000

Train Epoch: 2	 Loss:26.914143
Test -- Average loss:549.8263, Accuracy:0.000

Train Epoch: 3	 Loss:22.450313
Test -- Average loss:456.1255, Accuracy:0.000

Train Epoch: 4	 Loss:19.852806
Test -- Average loss:396.4222, Accuracy:0.000

Train Epoch: 5	 Loss:17.945302
Test -- Average loss:353.6947, Accuracy:0.000

Train Epoch: 6	 Loss:16.400667
Test -- Average loss:321.0599, Accuracy:0.000

Train Epoch: 7	 Loss:15.097411
Test -- Average loss:295.0797, Accuracy:0.000

Train Epoch: 8	 Loss:13.975291
Test -- Average loss:273.7791, Accuracy:0.000

Train Epoch: 9	 Loss:12.996857
Test -- Average loss:255.9104, Accuracy:0.000

Train Epoch: 10	 Loss:12.135696
Test -- Average loss:240.6351, Accuracy:0.000

Train Epoch: 11	 Loss:11.372006
Test -- Average loss:227.3737, Accuracy:0.000

Train Epoch: 12	 Loss:10.690572
Test -- Average loss:215.7158, Accuracy:0.000

Train Epoch: 13	 Loss:10.079422
Test -- Average loss:205.3625

Test -- Average loss:47.8652, Accuracy:0.000

Train Epoch: 113	 Loss:1.430801
Test -- Average loss:47.6223, Accuracy:0.000

Train Epoch: 114	 Loss:1.419078
Test -- Average loss:47.3838, Accuracy:0.000

Train Epoch: 115	 Loss:1.407567
Test -- Average loss:47.1498, Accuracy:0.000

Train Epoch: 116	 Loss:1.396263
Test -- Average loss:46.9200, Accuracy:0.000

Train Epoch: 117	 Loss:1.385161
Test -- Average loss:46.6943, Accuracy:0.000

Train Epoch: 118	 Loss:1.374254
Test -- Average loss:46.4728, Accuracy:0.000

Train Epoch: 119	 Loss:1.363537
Test -- Average loss:46.2552, Accuracy:0.000

Train Epoch: 120	 Loss:1.353005
Test -- Average loss:46.0414, Accuracy:0.000

Train Epoch: 121	 Loss:1.342653
Test -- Average loss:45.8314, Accuracy:0.000

Train Epoch: 122	 Loss:1.332476
Test -- Average loss:45.6251, Accuracy:0.000

Train Epoch: 123	 Loss:1.322469
Test -- Average loss:45.4223, Accuracy:0.000

Train Epoch: 124	 Loss:1.312629
Test -- Average loss:45.2230, Accuracy:0.000

Train Epoch: 125	 

Test -- Average loss:34.9417, Accuracy:0.000

Train Epoch: 220	 Loss:0.786811
Test -- Average loss:34.8826, Accuracy:0.000

Train Epoch: 221	 Loss:0.783763
Test -- Average loss:34.8241, Accuracy:0.000

Train Epoch: 222	 Loss:0.780744
Test -- Average loss:34.7661, Accuracy:0.000

Train Epoch: 223	 Loss:0.777754
Test -- Average loss:34.7087, Accuracy:0.000

Train Epoch: 224	 Loss:0.774792
Test -- Average loss:34.6518, Accuracy:0.000

Train Epoch: 225	 Loss:0.771858
Test -- Average loss:34.5954, Accuracy:0.000

Train Epoch: 226	 Loss:0.768951
Test -- Average loss:34.5395, Accuracy:0.000

Train Epoch: 227	 Loss:0.766071
Test -- Average loss:34.4841, Accuracy:0.000

Train Epoch: 228	 Loss:0.763217
Test -- Average loss:34.4292, Accuracy:0.000

Train Epoch: 229	 Loss:0.760390
Test -- Average loss:34.3748, Accuracy:0.000

Train Epoch: 230	 Loss:0.757590
Test -- Average loss:34.3209, Accuracy:0.000

Train Epoch: 231	 Loss:0.754814
Test -- Average loss:34.2674, Accuracy:0.000

Train Epoch: 232	 

Test -- Average loss:30.7452, Accuracy:0.000

Train Epoch: 326	 Loss:0.572452
Test -- Average loss:30.7192, Accuracy:0.000

Train Epoch: 327	 Loss:0.571119
Test -- Average loss:30.6933, Accuracy:0.000

Train Epoch: 328	 Loss:0.569796
Test -- Average loss:30.6676, Accuracy:0.000

Train Epoch: 329	 Loss:0.568480
Test -- Average loss:30.6421, Accuracy:0.000

Train Epoch: 330	 Loss:0.567173
Test -- Average loss:30.6167, Accuracy:0.000

Train Epoch: 331	 Loss:0.565874
Test -- Average loss:30.5915, Accuracy:0.000

Train Epoch: 332	 Loss:0.564583
Test -- Average loss:30.5665, Accuracy:0.000

Train Epoch: 333	 Loss:0.563300
Test -- Average loss:30.5416, Accuracy:0.000

Train Epoch: 334	 Loss:0.562025
Test -- Average loss:30.5169, Accuracy:0.000

Train Epoch: 335	 Loss:0.560757
Test -- Average loss:30.4924, Accuracy:0.000

Train Epoch: 336	 Loss:0.559498
Test -- Average loss:30.4680, Accuracy:0.000

Train Epoch: 337	 Loss:0.558247
Test -- Average loss:30.4438, Accuracy:0.000

Train Epoch: 338	 

In [123]:
f = Network(331, 150, 50, lr=0.001)
f.fit_pred(train_features_, train_labels_, test_features_, test_labels_ , Epochs=400, batch_size=50)

Train Epoch: 1	 Loss:51.143547
Test -- Average loss:762.8664, Accuracy:0.000

Train Epoch: 2	 Loss:30.885839
Test -- Average loss:540.5303, Accuracy:0.000

Train Epoch: 3	 Loss:21.831905
Test -- Average loss:424.1638, Accuracy:0.000

Train Epoch: 4	 Loss:17.065286
Test -- Average loss:351.8922, Accuracy:0.000

Train Epoch: 5	 Loss:14.155741
Test -- Average loss:303.2574, Accuracy:0.000

Train Epoch: 6	 Loss:12.174002
Test -- Average loss:268.5066, Accuracy:0.000

Train Epoch: 7	 Loss:10.710427
Test -- Average loss:242.3025, Accuracy:0.000

Train Epoch: 8	 Loss:9.566184
Test -- Average loss:221.6682, Accuracy:0.000

Train Epoch: 9	 Loss:8.638894
Test -- Average loss:204.8741, Accuracy:0.000

Train Epoch: 10	 Loss:7.870730
Test -- Average loss:190.8517, Accuracy:0.000

Train Epoch: 11	 Loss:7.225410
Test -- Average loss:178.9040, Accuracy:0.000

Train Epoch: 12	 Loss:6.677850
Test -- Average loss:168.5563, Accuracy:0.000

Train Epoch: 13	 Loss:6.209417
Test -- Average loss:159.4744, Accu

Train Epoch: 113	 Loss:1.274936
Test -- Average loss:35.1605, Accuracy:0.000

Train Epoch: 114	 Loss:1.270268
Test -- Average loss:35.0306, Accuracy:0.000

Train Epoch: 115	 Loss:1.265684
Test -- Average loss:34.9035, Accuracy:0.000

Train Epoch: 116	 Loss:1.261180
Test -- Average loss:34.7792, Accuracy:0.000

Train Epoch: 117	 Loss:1.256755
Test -- Average loss:34.6576, Accuracy:0.000

Train Epoch: 118	 Loss:1.252405
Test -- Average loss:34.5386, Accuracy:0.000

Train Epoch: 119	 Loss:1.248129
Test -- Average loss:34.4222, Accuracy:0.000

Train Epoch: 120	 Loss:1.243924
Test -- Average loss:34.3083, Accuracy:0.000

Train Epoch: 121	 Loss:1.239787
Test -- Average loss:34.1968, Accuracy:0.000

Train Epoch: 122	 Loss:1.235718
Test -- Average loss:34.0877, Accuracy:0.000

Train Epoch: 123	 Loss:1.231713
Test -- Average loss:33.9808, Accuracy:0.000

Train Epoch: 124	 Loss:1.227770
Test -- Average loss:33.8761, Accuracy:0.000

Train Epoch: 125	 Loss:1.223889
Test -- Average loss:33.7736, Ac

Test -- Average loss:28.9809, Accuracy:0.000

Train Epoch: 224	 Loss:0.988991
Test -- Average loss:28.9579, Accuracy:0.000

Train Epoch: 225	 Loss:0.987396
Test -- Average loss:28.9351, Accuracy:0.000

Train Epoch: 226	 Loss:0.985810
Test -- Average loss:28.9126, Accuracy:0.000

Train Epoch: 227	 Loss:0.984233
Test -- Average loss:28.8904, Accuracy:0.000

Train Epoch: 228	 Loss:0.982664
Test -- Average loss:28.8683, Accuracy:0.000

Train Epoch: 229	 Loss:0.981103
Test -- Average loss:28.8465, Accuracy:0.000

Train Epoch: 230	 Loss:0.979551
Test -- Average loss:28.8249, Accuracy:0.000

Train Epoch: 231	 Loss:0.978007
Test -- Average loss:28.8036, Accuracy:0.000

Train Epoch: 232	 Loss:0.976471
Test -- Average loss:28.7824, Accuracy:0.000

Train Epoch: 233	 Loss:0.974943
Test -- Average loss:28.7615, Accuracy:0.000

Train Epoch: 234	 Loss:0.973423
Test -- Average loss:28.7408, Accuracy:0.000

Train Epoch: 235	 Loss:0.971911
Test -- Average loss:28.7203, Accuracy:0.000

Train Epoch: 236	 

Test -- Average loss:27.3617, Accuracy:0.000

Train Epoch: 335	 Loss:0.851415
Test -- Average loss:27.3524, Accuracy:0.000

Train Epoch: 336	 Loss:0.850455
Test -- Average loss:27.3432, Accuracy:0.000

Train Epoch: 337	 Loss:0.849499
Test -- Average loss:27.3340, Accuracy:0.000

Train Epoch: 338	 Loss:0.848547
Test -- Average loss:27.3249, Accuracy:0.000

Train Epoch: 339	 Loss:0.847598
Test -- Average loss:27.3158, Accuracy:0.000

Train Epoch: 340	 Loss:0.846654
Test -- Average loss:27.3068, Accuracy:0.000

Train Epoch: 341	 Loss:0.845714
Test -- Average loss:27.2979, Accuracy:0.000

Train Epoch: 342	 Loss:0.844777
Test -- Average loss:27.2890, Accuracy:0.000

Train Epoch: 343	 Loss:0.843845
Test -- Average loss:27.2802, Accuracy:0.000

Train Epoch: 344	 Loss:0.842916
Test -- Average loss:27.2715, Accuracy:0.000

Train Epoch: 345	 Loss:0.841991
Test -- Average loss:27.2627, Accuracy:0.000

Train Epoch: 346	 Loss:0.841070
Test -- Average loss:27.2541, Accuracy:0.000

Train Epoch: 347	 

In [124]:
g = Network(331, 150, 80, lr=0.001)
g.fit_pred(train_features_, train_labels_, test_features_, test_labels_ , Epochs=400, batch_size=50)

Train Epoch: 1	 Loss:74.122969
Test -- Average loss:853.0652, Accuracy:0.000

Train Epoch: 2	 Loss:50.669278
Test -- Average loss:648.8172, Accuracy:0.000

Train Epoch: 3	 Loss:40.128332
Test -- Average loss:544.8266, Accuracy:0.000

Train Epoch: 4	 Loss:33.787477
Test -- Average loss:476.7982, Accuracy:0.000

Train Epoch: 5	 Loss:29.378949
Test -- Average loss:427.4905, Accuracy:0.000

Train Epoch: 6	 Loss:26.049725
Test -- Average loss:389.6408, Accuracy:0.000

Train Epoch: 7	 Loss:23.410439
Test -- Average loss:359.4663, Accuracy:0.000

Train Epoch: 8	 Loss:21.257098
Test -- Average loss:334.7074, Accuracy:0.000

Train Epoch: 9	 Loss:19.466297
Test -- Average loss:313.9376, Accuracy:0.000

Train Epoch: 10	 Loss:17.954910
Test -- Average loss:296.2058, Accuracy:0.000

Train Epoch: 11	 Loss:16.663465
Test -- Average loss:280.8469, Accuracy:0.000

Train Epoch: 12	 Loss:15.547993
Test -- Average loss:267.3803, Accuracy:0.000

Train Epoch: 13	 Loss:14.575280
Test -- Average loss:255.4491

Test -- Average loss:62.7972, Accuracy:0.000

Train Epoch: 116	 Loss:2.267123
Test -- Average loss:62.4599, Accuracy:0.000

Train Epoch: 117	 Loss:2.248484
Test -- Average loss:62.1283, Accuracy:0.000

Train Epoch: 118	 Loss:2.229937
Test -- Average loss:61.8023, Accuracy:0.000

Train Epoch: 119	 Loss:2.211474
Test -- Average loss:61.4817, Accuracy:0.000

Train Epoch: 120	 Loss:2.193089
Test -- Average loss:61.1663, Accuracy:0.000

Train Epoch: 121	 Loss:2.174778
Test -- Average loss:60.8562, Accuracy:0.000

Train Epoch: 122	 Loss:2.156535
Test -- Average loss:60.5510, Accuracy:0.000

Train Epoch: 123	 Loss:2.138357
Test -- Average loss:60.2509, Accuracy:0.000

Train Epoch: 124	 Loss:2.120241
Test -- Average loss:59.9555, Accuracy:0.000

Train Epoch: 125	 Loss:2.102183
Test -- Average loss:59.6648, Accuracy:0.000

Train Epoch: 126	 Loss:2.084183
Test -- Average loss:59.3788, Accuracy:0.000

Train Epoch: 127	 Loss:2.066239
Test -- Average loss:59.0972, Accuracy:0.000

Train Epoch: 128	 

Test -- Average loss:43.4333, Accuracy:0.000

Train Epoch: 230	 Loss:1.058863
Test -- Average loss:43.3500, Accuracy:0.000

Train Epoch: 231	 Loss:1.055009
Test -- Average loss:43.2675, Accuracy:0.000

Train Epoch: 232	 Loss:1.051194
Test -- Average loss:43.1857, Accuracy:0.000

Train Epoch: 233	 Loss:1.047417
Test -- Average loss:43.1046, Accuracy:0.000

Train Epoch: 234	 Loss:1.043678
Test -- Average loss:43.0243, Accuracy:0.000

Train Epoch: 235	 Loss:1.039975
Test -- Average loss:42.9447, Accuracy:0.000

Train Epoch: 236	 Loss:1.036308
Test -- Average loss:42.8658, Accuracy:0.000

Train Epoch: 237	 Loss:1.032676
Test -- Average loss:42.7876, Accuracy:0.000

Train Epoch: 238	 Loss:1.029078
Test -- Average loss:42.7101, Accuracy:0.000

Train Epoch: 239	 Loss:1.025514
Test -- Average loss:42.6332, Accuracy:0.000

Train Epoch: 240	 Loss:1.021982
Test -- Average loss:42.5571, Accuracy:0.000

Train Epoch: 241	 Loss:1.018483
Test -- Average loss:42.4816, Accuracy:0.000

Train Epoch: 242	 

Test -- Average loss:37.4257, Accuracy:0.000

Train Epoch: 338	 Loss:0.775643
Test -- Average loss:37.3891, Accuracy:0.000

Train Epoch: 339	 Loss:0.773802
Test -- Average loss:37.3527, Accuracy:0.000

Train Epoch: 340	 Loss:0.771972
Test -- Average loss:37.3166, Accuracy:0.000

Train Epoch: 341	 Loss:0.770150
Test -- Average loss:37.2807, Accuracy:0.000

Train Epoch: 342	 Loss:0.768338
Test -- Average loss:37.2451, Accuracy:0.000

Train Epoch: 343	 Loss:0.766535
Test -- Average loss:37.2096, Accuracy:0.000

Train Epoch: 344	 Loss:0.764742
Test -- Average loss:37.1744, Accuracy:0.000

Train Epoch: 345	 Loss:0.762957
Test -- Average loss:37.1394, Accuracy:0.000

Train Epoch: 346	 Loss:0.761182
Test -- Average loss:37.1047, Accuracy:0.000

Train Epoch: 347	 Loss:0.759416
Test -- Average loss:37.0701, Accuracy:0.000

Train Epoch: 348	 Loss:0.757658
Test -- Average loss:37.0358, Accuracy:0.000

Train Epoch: 349	 Loss:0.755909
Test -- Average loss:37.0016, Accuracy:0.000

Train Epoch: 350	 

In [110]:
d = Network(331, 100, 50, lr=0.001)
d.fit_pred(train_features_, train_labels_, test_features_, test_labels_ , Epochs=300, batch_size=50)
d.lr=0.005
d.fit_pred(train_features_, train_labels_, test_features_, test_labels_ , Epochs=100, batch_size=50)

Train Epoch: 1	 Loss:82.884668
Test -- Average loss:767.8225, Accuracy:0.000

Train Epoch: 2	 Loss:43.835110
Test -- Average loss:525.7028, Accuracy:0.000

Train Epoch: 3	 Loss:30.647347
Test -- Average loss:417.3771, Accuracy:0.000

Train Epoch: 4	 Loss:23.700576
Test -- Average loss:351.5090, Accuracy:0.000

Train Epoch: 5	 Loss:19.314408
Test -- Average loss:305.8158, Accuracy:0.000

Train Epoch: 6	 Loss:16.290149
Test -- Average loss:271.9236, Accuracy:0.000

Train Epoch: 7	 Loss:14.088424
Test -- Average loss:245.7075, Accuracy:0.000

Train Epoch: 8	 Loss:12.423601
Test -- Average loss:224.8030, Accuracy:0.000

Train Epoch: 9	 Loss:11.126589
Test -- Average loss:207.7280, Accuracy:0.000

Train Epoch: 10	 Loss:10.089764
Test -- Average loss:193.5001, Accuracy:0.000

Train Epoch: 11	 Loss:9.241803
Test -- Average loss:181.4422, Accuracy:0.000

Train Epoch: 12	 Loss:8.534230
Test -- Average loss:171.0737, Accuracy:0.000

Train Epoch: 13	 Loss:7.933414
Test -- Average loss:162.0452, A

Test -- Average loss:42.9569, Accuracy:0.000

Train Epoch: 120	 Loss:1.544498
Test -- Average loss:42.8186, Accuracy:0.000

Train Epoch: 121	 Loss:1.536817
Test -- Average loss:42.6826, Accuracy:0.000

Train Epoch: 122	 Loss:1.529217
Test -- Average loss:42.5488, Accuracy:0.000

Train Epoch: 123	 Loss:1.521694
Test -- Average loss:42.4173, Accuracy:0.000

Train Epoch: 124	 Loss:1.514250
Test -- Average loss:42.2879, Accuracy:0.000

Train Epoch: 125	 Loss:1.506880
Test -- Average loss:42.1606, Accuracy:0.000

Train Epoch: 126	 Loss:1.499586
Test -- Average loss:42.0353, Accuracy:0.000

Train Epoch: 127	 Loss:1.492365
Test -- Average loss:41.9120, Accuracy:0.000

Train Epoch: 128	 Loss:1.485216
Test -- Average loss:41.7906, Accuracy:0.000

Train Epoch: 129	 Loss:1.478139
Test -- Average loss:41.6711, Accuracy:0.000

Train Epoch: 130	 Loss:1.471131
Test -- Average loss:41.5535, Accuracy:0.000

Train Epoch: 131	 Loss:1.464193
Test -- Average loss:41.4377, Accuracy:0.000

Train Epoch: 132	 

Test -- Average loss:34.7418, Accuracy:0.000

Train Epoch: 239	 Loss:0.999880
Test -- Average loss:34.7088, Accuracy:0.000

Train Epoch: 240	 Loss:0.997385
Test -- Average loss:34.6762, Accuracy:0.000

Train Epoch: 241	 Loss:0.994910
Test -- Average loss:34.6438, Accuracy:0.000

Train Epoch: 242	 Loss:0.992456
Test -- Average loss:34.6118, Accuracy:0.000

Train Epoch: 243	 Loss:0.990021
Test -- Average loss:34.5800, Accuracy:0.000

Train Epoch: 244	 Loss:0.987606
Test -- Average loss:34.5486, Accuracy:0.000

Train Epoch: 245	 Loss:0.985211
Test -- Average loss:34.5174, Accuracy:0.000

Train Epoch: 246	 Loss:0.982834
Test -- Average loss:34.4866, Accuracy:0.000

Train Epoch: 247	 Loss:0.980477
Test -- Average loss:34.4560, Accuracy:0.000

Train Epoch: 248	 Loss:0.978138
Test -- Average loss:34.4257, Accuracy:0.000

Train Epoch: 249	 Loss:0.975818
Test -- Average loss:34.3957, Accuracy:0.000

Train Epoch: 250	 Loss:0.973516
Test -- Average loss:34.3660, Accuracy:0.000

Train Epoch: 251	 

Train Epoch: 50	 Loss:2.548615
Test -- Average loss:47.8503, Accuracy:0.000

Train Epoch: 51	 Loss:2.449330
Test -- Average loss:46.6241, Accuracy:0.000

Train Epoch: 52	 Loss:2.366754
Test -- Average loss:45.4906, Accuracy:0.000

Train Epoch: 53	 Loss:2.297375
Test -- Average loss:44.4557, Accuracy:0.000

Train Epoch: 54	 Loss:2.237041
Test -- Average loss:43.5131, Accuracy:0.000

Train Epoch: 55	 Loss:2.182785
Test -- Average loss:42.6527, Accuracy:0.000

Train Epoch: 56	 Loss:2.133108
Test -- Average loss:41.8677, Accuracy:0.000

Train Epoch: 57	 Loss:2.087609
Test -- Average loss:41.1543, Accuracy:0.000

Train Epoch: 58	 Loss:2.046271
Test -- Average loss:40.5061, Accuracy:0.000

Train Epoch: 59	 Loss:2.008989
Test -- Average loss:39.9144, Accuracy:0.000

Train Epoch: 60	 Loss:1.975550
Test -- Average loss:39.3711, Accuracy:0.000

Train Epoch: 61	 Loss:1.945678
Test -- Average loss:38.8696, Accuracy:0.000

Train Epoch: 62	 Loss:1.919064
Test -- Average loss:38.4046, Accuracy:0.000


In [103]:
d.lr=0.005
d.fit_pred(train_features_, train_labels_, test_features_, test_labels_ , Epochs=50, batch_size=50)

Train Epoch: 1	 Loss:0.741119
Test -- Average loss:6.7168, Accuracy:0.000

Train Epoch: 2	 Loss:0.737415
Test -- Average loss:6.7036, Accuracy:0.000

Train Epoch: 3	 Loss:0.735087
Test -- Average loss:6.6897, Accuracy:0.000

Train Epoch: 4	 Loss:0.732981
Test -- Average loss:6.6757, Accuracy:0.000

Train Epoch: 5	 Loss:0.730958
Test -- Average loss:6.6619, Accuracy:0.000

Train Epoch: 6	 Loss:0.728997
Test -- Average loss:6.6481, Accuracy:0.000

Train Epoch: 7	 Loss:0.727086
Test -- Average loss:6.6346, Accuracy:0.000

Train Epoch: 8	 Loss:0.725221
Test -- Average loss:6.6212, Accuracy:0.000

Train Epoch: 9	 Loss:0.723397
Test -- Average loss:6.6080, Accuracy:0.000

Train Epoch: 10	 Loss:0.721610
Test -- Average loss:6.5950, Accuracy:0.000

Train Epoch: 11	 Loss:0.719857
Test -- Average loss:6.5823, Accuracy:0.000

Train Epoch: 12	 Loss:0.718136
Test -- Average loss:6.5697, Accuracy:0.000

Train Epoch: 13	 Loss:0.716445
Test -- Average loss:6.5574, Accuracy:0.000

Train Epoch: 14	 Loss

In [105]:
d.lr=0.004
d.fit_pred(train_features_, train_labels_, test_features_, test_labels_ , Epochs=50, batch_size=50)

Train Epoch: 1	 Loss:0.629034
Test -- Average loss:6.0594, Accuracy:0.000

Train Epoch: 2	 Loss:0.627766
Test -- Average loss:6.0563, Accuracy:0.000

Train Epoch: 3	 Loss:0.626789
Test -- Average loss:6.0530, Accuracy:0.000

Train Epoch: 4	 Loss:0.625863
Test -- Average loss:6.0497, Accuracy:0.000

Train Epoch: 5	 Loss:0.624950
Test -- Average loss:6.0464, Accuracy:0.000

Train Epoch: 6	 Loss:0.624045
Test -- Average loss:6.0432, Accuracy:0.000

Train Epoch: 7	 Loss:0.623145
Test -- Average loss:6.0400, Accuracy:0.000

Train Epoch: 8	 Loss:0.622250
Test -- Average loss:6.0368, Accuracy:0.000

Train Epoch: 9	 Loss:0.621360
Test -- Average loss:6.0337, Accuracy:0.000

Train Epoch: 10	 Loss:0.620474
Test -- Average loss:6.0305, Accuracy:0.000

Train Epoch: 11	 Loss:0.619593
Test -- Average loss:6.0275, Accuracy:0.000

Train Epoch: 12	 Loss:0.618715
Test -- Average loss:6.0244, Accuracy:0.000

Train Epoch: 13	 Loss:0.617840
Test -- Average loss:6.0215, Accuracy:0.000

Train Epoch: 14	 Loss

In [125]:
# 得出预测
preds = np.exp(f.predict_(test_features))

In [126]:
# 将test_data的SalePrice列填入预测数据
test_data['SalePrice'] = pd.Series(preds.reshape(1, -1)[0])

In [127]:
# 转化为csv文件
submission = pd.concat([test_data['Id'], test_data['SalePrice']], axis=1)
submission.to_csv('submission.csv', index=False)